In [1]:
from nanome.service import PluginService
from nanome.interface import PluginInstanceRedisInterface

# Set up redis credentials
redis_host = 'redis'
redis_port = 6379

# When your PluginService is running, you can get your channel value from the Logs, or from the query parameter in your open browser.
# Update this value to match that, so that your commands will run against your live workspace.
redis_channel = '6bc0d46f-b622-45eb-8623-886216c906bc'

plugin_instance = PluginInstanceRedisInterface(redis_host, redis_port, redis_channel)
print('Hello')

Hello


In [ ]:
comps = plugin_instance.request_complex_list()
print(comp)


Sending request_complex_list Request to Redis Channel 6bc0d46f-b622-45eb-8623-886216c906bc


In [ ]:
# Changing complex color
# print("changing comp color")
# plugin_instance.cycle_color([comp.index])

In [ ]:
# neo4j setup
from py2neo import Graph, Node, Relationship

neo4j_uri = 'bolt://neo4j:7687/'
username = 'neo4j'
password = 'F8dwclytew'
graph = Graph(neo4j_uri, auth=(username, password))


In [ ]:
with open("assets/neo4j_query.txt") as f:
    query_string = f.read()
results = graph.run(query_string)
data = results.data()

In [2]:
from nanome.api.structure import Atom

atom_list = []
for record in data:
    node = record['a']
    print(next(iter(node.values())))
    continue
    new_atom = Atom(name=next(iter(node.values())))
    atom_list.append()

NameError: name 'data' is not defined

In [ ]:
plugin = PluginInstanceRedisInterface(redis_host, redis_port)

In [ ]:
plugin.update_structures_deep(atom_list)

In [ ]:
atom = Atom()
atom.name = 'Test Atom'

plugin_instance.update_structures_deep([atom])